# Notebook with some different code used in development

## Downloading images

In [6]:
# read json

import json
import requests

url = "https://raw.githubusercontent.com/arthurdjn/img2poem-pytorch/master/data/poems/multim_poem.json"
r = requests.get(url).text
images = json.loads(r)

In [8]:
images[0]

{'id': 0,
 'image_url': 'https://farm2.staticflickr.com/1086/1002051357_0e9162423e.jpg',
 'poem': 'what is lovely never dies\nbut passes into other loveliness\nstar-dust or sea-foam flower or winged air'}

In [15]:
with open(f"{images[0]['id']}.jpg", 'wb') as handle:
    response = requests.get(images[0]["image_url"], stream=True)
    for block in response.iter_content(1024):
        handle.write(block)

## Image-to-text

https://huggingface.co/nlpconnect/vit-gpt2-image-captioning

In [2]:
from transformers import pipeline

image_to_text = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")

image_to_text("https://ankur3107.github.io/assets/images/image-captioning-example.png")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
c:\Users\lukas\miniconda3\envs\nlp_venv\lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
c:\Users\lukas\miniconda3\envs\nlp_venv\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` 

[{'generated_text': 'a soccer game with a player jumping to catch the ball '}]

In [3]:
image_to_text(["data/poem_images/0.jpg", "data/poem_images/1.jpg"])

c:\Users\lukas\miniconda3\envs\nlp_venv\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[[{'generated_text': 'two pink flowers in a flower pot '}],
 [{'generated_text': 'a flower arrangement in a vase with other flowers '}]]

In [18]:
image_to_text("https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fupload.wikimedia.org%2Fwikipedia%2Fcommons%2Fd%2Fda%2FPurple_flower_(4764445139).jpg&f=1&nofb=1&ipt=8c21a5c16ce190f8a486d6df01c56650fa7ae77bd8f0a5b790f141bb94b5eabd&ipo=images")

c:\Users\lukas\miniconda3\envs\nlp_venv\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'a flower in a blue vase with a blue background '}]

## Caption poem data set

In [19]:
import json
import time
data = json.load(open("../data/multim_poem.json", "r"))
urls = [item["image_url"] for item in data]

In [22]:
from transformers import pipeline
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
image_to_text = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning", device=device)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
C:\Users\lukas\AppData\Roaming\Python\Python310\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [20]:
for i in range(5):
    t0 = time.time()
    data[i]["caption"] = image_to_text(data[i]["image_url"])[0]["generated_text"]
    print(f"Generated caption for id {data[i]['id']} in {time.time()-t0:.5f} s")

C:\Users\lukas\AppData\Roaming\Python\Python310\site-packages\transformers\generation\utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated caption for id 0 in 2.5103816986083984 s
Generated caption for id 1 in 2.8679747581481934 s
Generated caption for id 2 in 3.078000068664551 s
Generated caption for id 3 in 2.6855885982513428 s
Generated caption for id 4 in 2.506525993347168 s


In [24]:
for i in range(5):
    t0 = time.time()
    data[i]["caption"] = image_to_text(data[i]["image_url"])[0]["generated_text"]
    print(f"Generated caption for id {data[i]['id']} in {time.time()-t0:.5f} s")

Generated caption for id 0 in 1.56683 s
Generated caption for id 1 in 1.62027 s
Generated caption for id 2 in 1.54528 s
Generated caption for id 3 in 1.28317 s
Generated caption for id 4 in 1.33454 s


In [25]:
data

[{'id': 0,
  'image_url': 'https://farm2.staticflickr.com/1086/1002051357_0e9162423e.jpg',
  'poem': 'what is lovely never dies\nbut passes into other loveliness\nstar-dust or sea-foam flower or winged air',
  'caption': 'two pink flowers in a flower pot '},
 {'id': 1,
  'image_url': 'https://farm8.staticflickr.com/7434/10024691123_ba6ba177c6.jpg',
  'poem': "sods on the dugout begin to be fledged\nwith fine green feathers of yarrow\ni once glimpsed fairies by squinting the leaves\nbut now i scry nothing but sorrow\nfor i have seen milfoils grow under a hedge\ni've seen them stand up on a barrow\nbut what shall i think when they burst into bloom\nthe flowers stained mustard-gas yellow\nthe sun and the wind are drying the mud\nbut the mortar and shell they will harrow",
  'caption': 'a flower arrangement in a vase with other flowers '},
 {'id': 2,
  'image_url': 'https://farm1.staticflickr.com/19/100255672_97af849941.jpg',
  'poem': 'one must have the mind of winter\nto regard the front